#  Install and Import Dependencies

In [1]:
!pip install mediapipe opencv-python


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

#  Determining Joints

<img src="https://i.imgur.com/3j8BPdc.png" style="height:300px" >

#  Calculate Angles

Pose-Based Fitness Tracker project to calculate the angle between body joints, such as the elbow, shoulder, and wrist, during bicep curls.

In [3]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

#  Curl Counter

### How Pose Landmarks Detection Works in MediaPipe

MediaPipe's **Pose Landmarks Detection** system is a real-time pose estimation technique that predicts the position of key body joints (or landmarks) in images or video frames. It uses a combination of machine learning and deep learning techniques. Here’s a breakdown of how it works:

---

### 1. Input Image Processing
- The input to the pose detection system is an image (or a sequence of frames in a video) that is preprocessed.
- The image is resized and normalized to fit the model's input size requirements.

### 2. Pose Detection Model
- MediaPipe utilizes a deep learning model, typically a **Convolutional Neural Network (CNN)**, that has been trained on a large dataset of human poses.
- The model first performs **region detection**, identifying whether a person (or parts of the body) is visible in the image.
- It creates a **bounding box** around the detected person.

### 3. Landmark Prediction
- Once a person is detected, the system predicts **33 body landmarks**, which include key points like the:
  - Shoulders
  - Elbows
  - Wrists
  - Hips
  - Knees
  - Ankles
  
  Each landmark point is represented by 3 coordinates:
  - **x**: Horizontal position in the image.
  - **y**: Vertical position in the image.
  - **z**: Depth (indicating how far the point is from the camera).

### 4. Model Architecture
- MediaPipe’s Pose model employs a **two-step process**:
  1. **Detector**: First, a lightweight detector identifies the region of interest (person) in the image.
  2. **Pose Estimator**: Then, a more refined network predicts the precise locations of landmarks within the detected region.

### 5. Coordinate Transformation
- The model outputs normalized coordinates, where **x** and **y** are relative to the image dimensions (ranging from 0 to 1). The **z** coordinate is relative to the depth of the body parts, which helps to understand 3D movements.

### 6. Real-Time Processing
- One of MediaPipe’s strengths is its **efficiency**. The system is optimized to perform landmark detection in real-time, even on mobile devices, by balancing accuracy and computational cost.

### 7. Tracking Across Frames
- When dealing with videos, MediaPipe continuously tracks the detected landmarks across frames, allowing for **smooth pose estimation** and **real-time feedback**. This is useful for applications like fitness tracking, dance movement analysis, or even augmented reality (AR).

---

### Applications of Pose Landmarks Detection
- **Fitness tracking**: To track body movements and count exercise repetitions.
- **Gesture recognition**: To recognize gestures or specific movements.
- **Animation and gaming**: To capture human motion for character animation.
- **Health and rehabilitation**: To analyze posture and body mechanics.

---

This detection process allows for capturing detailed, real-time body movement, enabling a wide range of applications.


In [4]:
cap = cv2.VideoCapture(0)

# Curl counter variables
counter = 0 
stage = None

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            
            # Calculate angle
            angle = calculate_angle(shoulder, elbow, wrist)
            
            # Visualize angle
            cv2.putText(image, str(angle), 
                           tuple(np.multiply(elbow, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            # Curl counter logic
            if angle > 160:
                stage = "down"
            if angle < 30 and stage =='down':
                stage="up"
                counter +=1
                print(counter)
                       
        except:
            pass
        
        # Render curl counter
        # Setup status box
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
        
        # Rep data
        cv2.putText(image, 'REPS', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Stage data
        cv2.putText(image, 'STAGE', (65,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stage, 
                    (60,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

C:\Users\vikas\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1
2
3
4
5
6
7
8


### Pose Landmarks Detection (Human Pose)

If you're using pose detection (like in your case), MediaPipe uses a trained neural network to detect key **landmarks** on the human body. These landmarks represent joints or key points like the shoulder, elbow, wrist, hip, etc. It predicts the positions of these points relative to the image dimensions.

#### Pose Landmarks Model
This model is based on deep learning and is optimized for efficiency, allowing real-time processing. It outputs the **x, y, and z coordinates** for multiple body points.


### 1. Importing Required Libraries
In this section, we import essential libraries:

- `cv2`: OpenCV for video capture and image processing.
- `numpy`: For mathematical operations.
- `mediapipe`: For pose estimation.

### 2. Helper Function for Angle Calculation
This function calculates the angle between three points (shoulder, elbow, wrist). 
The angle is crucial for detecting bicep curls in the real-time video.

### 3. Pose Estimation and Curl Counter Setup
This block initializes the video capture from the webcam and sets up the MediaPipe pose estimation model.
It also sets up variables for counting bicep curls (`counter`) and tracking the movement stage (`stage`).

### 4. Main Loop for Curl Detection
The main loop does the following:

- Continuously captures frames from the webcam.
- Converts frames to RGB for pose detection.
- Uses MediaPipe to detect body landmarks (shoulder, elbow, wrist).
- Calculates the angle between these landmarks.
- Implements logic to count curls when the arm goes from a "down" to "up" position.
- Displays the number of reps and the current stage ("up" or "down") on the video feed.

### 5. Closing Video Capture
After pressing the 'q' key, the video capture ends, and the window closes.


### Comparison: MediaPipe vs. OpenPose

**MediaPipe** and **OpenPose** are both popular frameworks for pose estimation and related tasks. Below is a comparison highlighting why MediaPipe is often considered more efficient:

---

### 1. **Real-Time Performance**

- **MediaPipe**:
  - **Optimized for Real-Time**: MediaPipe is designed specifically for real-time performance, making it suitable for applications like live video processing.
  - **Low Latency**: It achieves low-latency processing through optimized models and pipelines, which is crucial for interactive applications.

- **OpenPose**:
  - **Higher Latency**: OpenPose, while highly accurate, often has higher latency due to its more complex models and processing requirements.
  - **Performance**: It requires significant computational resources, which may affect real-time performance, especially on lower-end devices.

---

### 2. **Resource Efficiency**

- **MediaPipe**:
  - **Lightweight Models**: MediaPipe uses lightweight models that are optimized to run efficiently on a wide range of devices, including mobile and web.
  - **Adaptive Resource Usage**: It adapts to the hardware capabilities, ensuring efficient use of available resources.

- **OpenPose**:
  - **Resource-Intensive**: OpenPose models are generally more resource-intensive, requiring powerful GPUs and high computational power.
  - **Hardware Dependency**: It performs best on high-end GPUs, which may not be available in all devices.

---

### 3. **Cross-Platform Support**

- **MediaPipe**:
  - **Versatile Deployment**: Supports multiple platforms, including mobile (iOS, Android), web (via WebAssembly), and desktop.
  - **Unified Framework**: Provides a consistent framework across different environments, making it easier to deploy and integrate.

- **OpenPose**:
  - **Limited Cross-Platform Support**: Primarily designed for desktop environments, with more complex setup and less support for mobile or web platforms.
  - **Platform Constraints**: Deployment on different platforms can be challenging due to its heavy dependencies and computational requirements.

---

### 4. **Ease of Use and Integration**

- **MediaPipe**:
  - **High-Level APIs**: Offers high-level APIs and tools that simplify the development process.
  - **Pre-Built Solutions**: Comes with pre-trained models for various tasks, reducing the need for extensive custom model training.

- **OpenPose**:
  - **Complex Setup**: Requires more complex setup and configuration, including dependencies and model training.
  - **Development Effort**: Integrating OpenPose into applications can be more challenging due to its complexity.

---

### 5. **Customizability**

- **MediaPipe**:
  - **Modular Design**: Allows for the creation of custom pipelines and easy integration with other systems.
  - **Flexible Pipelines**: Users can customize and extend existing models to fit specific use cases.

- **OpenPose**:
  - **Less Modular**: While powerful, OpenPose's architecture is less modular, and customizing or extending the model can be more involved.

---

### 6. **Community and Support**

- **MediaPipe**:
  - **Active Development**: Maintained and developed by Google with active updates and improvements.
  - **Community Contributions**: Benefits from contributions and feedback from a large community.

- **OpenPose**:
  - **Active but Limited**: Maintained by the Carnegie Mellon Perceptual Computing Lab with a dedicated community but less frequent updates compared to MediaPipe.

---

### Summary

**MediaPipe** is often considered more efficient due to its real-time performance, resource efficiency, cross-platform support, ease of use, and modularity. While **OpenPose** offers high accuracy and comprehensive features, its higher computational requirements and complexity can make it less suitable for real-time applications and less resource-constrained environments.

---

Both frameworks have their strengths and are suitable for different applications. The choice between them depends on specific needs, such as the importance of real-time performance versus accuracy, and the available hardware resources.


## Project Challenges and Solutions

### Challenges Addressed

1. **Real-Time Pose Detection**
   - **Problem:** Efficiently detecting and tracking key landmarks on the human body in real-time using video feed.
   - **Solution:** Utilized MediaPipe's Pose model, which is optimized for real-time processing. This model provides accurate detection of key points such as shoulders, elbows, and wrists, which are crucial for tracking exercises.

2. **Accurate Angle Calculation**
   - **Problem:** Determining the angle between body joints (e.g., shoulder, elbow, and wrist) to count repetitions of exercises like bicep curls.
   - **Solution:** Implemented a `calculate_angle` function to compute the angle formed by three points. This function uses trigonometric calculations to measure the angle and adjust it if it exceeds 180 degrees.

3. **Rep Counting and Stage Detection**
   - **Problem:** Counting the number of exercise repetitions and detecting different stages of the exercise (e.g., upward and downward phases of a curl).
   - **Solution:** Applied angle thresholds to distinguish between different stages of the exercise. The counter is incremented when the exercise transitions from the downward to the upward phase.

4. **Handling Video Capture and Display**
   - **Problem:** Managing video capture from a webcam and displaying results in a user-friendly manner.
   - **Solution:** Used OpenCV to capture video frames and display the real-time feed with overlaid text and visualizations. Ensured proper handling of video frames and GUI rendering.

5. **User Interface for Interaction**
   - **Problem:** Providing a user-friendly interface for starting and stopping the workout.
   - **Solution:** Developed a simple GUI using Tkinter, allowing users to start and stop the workout session. Included buttons to control the workout and display feedback messages.

6. **Thread Management and Graceful Termination**
   - **Problem:** Ensuring the workout session can be stopped gracefully and without leaving resources open.
   - **Solution:** Implemented threading with a stop flag to manage the workout session. The flag signals the workout thread to terminate, and the thread is joined to ensure clean exit.

7. **Data Recording and Feedback**
   - **Problem:** Saving workout data and providing feedback to the user.
   - **Solution:** Created a log file to record workout details, including the number of curls and timestamps. Added message boxes to inform users when the workout starts, stops, and completes.

### Summary

Your project effectively addressed the challenges of real-time pose detection, angle calculation, rep counting, and user interaction. By leveraging MediaPipe for pose detection and OpenCV for video handling, along with a Tkinter-based GUI for user control, you created a robust fitness tracker that provides real-time feedback and ensures a seamless user experience.


## Applications of the Fitness Tracking Project

### 1. **Personal Fitness Training**
   - **Description:** Provides individuals with real-time feedback on their exercise form and repetition count, helping them to perform exercises correctly and effectively. Ideal for users looking to improve their workout routines at home.

### 2. **Physical Rehabilitation**
   - **Description:** Assists patients undergoing physical rehabilitation by monitoring their exercise progress and ensuring proper technique. Useful for physiotherapists to track patient adherence to prescribed exercises.

### 3. **Sports Coaching**
   - **Description:** Enables coaches to analyze athletes' performance by providing detailed feedback on exercise form and counting repetitions. Helps in optimizing training regimens and preventing injuries.

### 4. **Fitness App Integration**
   - **Description:** Can be integrated into mobile or desktop fitness applications to offer users an interactive way to track their workouts and monitor their progress. Enhances user engagement by providing real-time exercise feedback.

### 5. **Virtual Personal Training**
   - **Description:** Supports virtual personal training sessions by allowing trainers to monitor and guide clients remotely. Facilitates personalized training plans and real-time corrections.

### 6. **Gym Management Systems**
   - **Description:** Can be implemented in gym management systems to offer automated feedback and performance tracking for gym-goers. Provides valuable data for gym operators to offer enhanced services and support.

### 7. **Research and Development**
   - **Description:** Useful for researchers studying exercise biomechanics, human motion analysis, and machine learning applications in fitness. Provides a foundation for developing advanced exercise tracking technologies.

### 8. **Health Monitoring Devices**
   - **Description:** Can be incorporated into wearable health monitoring devices to offer exercise tracking and performance analysis as part of a comprehensive health monitoring solution.

### 9. **Educational Tools**
   - **Description:** Serves as an educational tool for teaching students about computer vision, pose estimation, and fitness technology. Provides hands-on experience with real-world applications of these technologies.

### Summary

The fitness tracking project has diverse applications across personal fitness, rehabilitation, coaching, and technology integration. Its ability to provide real-time feedback and track exercise performance makes it a valuable tool for improving fitness outcomes and supporting various health and fitness-related initiatives.
